<a href="https://colab.research.google.com/github/sauryanshu55/CryptoPlay-ETF/blob/main/Asset_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.optimize import minimize


Define Constants

In [21]:
equities_ticker = "ESPO"
crypto_tickers = ["BTC-USD", "ETH-USD"]
start_date = "2019-12-14"
end_date = "2024-12-14"
risk_free_rate=0.03

Fetch Data

In [22]:
def fetch_data(tickers, start, end):
    data = yf.download(tickers, start=start, end=end)["Adj Close"]
    return data


Calculate Annualized Returns

In [23]:
def calculate_annualized_metrics(returns):
    mean_returns = returns.mean() * 252  # Annualize returns
    cov_matrix = returns.cov() * 252    # Annualize covariance
    return mean_returns, cov_matrix

Calculate Portfolio performance

In [24]:
def portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate=risk_free_rate):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
    return portfolio_return, portfolio_volatility, sharpe_ratio

Optimize for Sharpe Ratio

In [25]:
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=risk_free_rate):
    _, _, sharpe_ratio = portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate)
    return -sharpe_ratio

In [26]:
def optimize_portfolio(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    bounds = tuple((0, 1) for _ in range(num_assets))
    initial_weights = num_assets * [1. / num_assets]

    result = minimize(neg_sharpe_ratio, initial_weights, args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

Bringing it all together

In [27]:
# Fetch data
equities_data = fetch_data([equities_ticker], start_date, end_date)
crypto_data = fetch_data(crypto_tickers, start_date, end_date)

# Calculate returns
equities_returns = equities_data.pct_change().dropna()
crypto_returns = crypto_data.pct_change().dropna()

# Calculate annualized metrics
equities_mean_returns, equities_cov_matrix = calculate_annualized_metrics(equities_returns)
crypto_mean_returns, crypto_cov_matrix = calculate_annualized_metrics(crypto_returns)

# Optimize individual portfolios
equities_weights = optimize_portfolio(equities_mean_returns, equities_cov_matrix)
crypto_weights = optimize_portfolio(crypto_mean_returns, crypto_cov_matrix)

# Combine into asset class weights
equities_portfolio_return = np.dot(equities_weights, equities_mean_returns)
equities_portfolio_volatility = np.sqrt(np.dot(equities_weights.T, np.dot(equities_cov_matrix, equities_weights)))

crypto_portfolio_return = np.dot(crypto_weights, crypto_mean_returns)
crypto_portfolio_volatility = np.sqrt(np.dot(crypto_weights.T, np.dot(crypto_cov_matrix, crypto_weights)))

# Calculate Sharpe ratio-based allocation
total_volatility = equities_portfolio_volatility + crypto_portfolio_volatility
equities_allocation = equities_portfolio_volatility / total_volatility
crypto_allocation = crypto_portfolio_volatility / total_volatility

# Combine returns and volatilities for total portfolio
total_return = (
  equities_allocation * equities_portfolio_return +
  crypto_allocation * crypto_portfolio_return
)

# Calculate covariance between equities and crypto returns
combined_returns = pd.concat([equities_returns, crypto_returns], axis=1)
covariance_matrix = combined_returns.cov() * 252  # Annualize covariance
equities_crypto_cov = covariance_matrix.iloc[0, 1]

total_volatility = np.sqrt(
  (equities_allocation ** 2 * equities_portfolio_volatility ** 2) +
  (crypto_allocation ** 2 * crypto_portfolio_volatility ** 2) +
  (2 * equities_allocation * crypto_allocation * equities_crypto_cov)
)

# Calculate total portfolio Sharpe ratio
total_sharpe_ratio = (total_return - risk_free_rate) / total_volatility


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed


**Total Metrics**

In [28]:
print("Total Portfolio Sharpe Ratio:", total_sharpe_ratio)
print("Total Expected Return:", total_return)
print("Total Volatility: ", total_volatility)

Total Portfolio Sharpe Ratio: 1.0278856245409305
Total Expected Return: 0.49818466010906154
Total Volatility:  0.45548322588727713


**Asset Metrics**

In [30]:
print("Crypto Allocation (Asset Class):", crypto_allocation)
print("Equities Weights:", equities_weights)
print("Crypto Weights:", crypto_weights)
print("Equities Portfolio return", equities_portfolio_return)
print("Crypto Portfolio return", crypto_portfolio_return)
print("Equities Portfolio Volatility", equities_portfolio_volatility)
print("Crypto Portfolio Volatility", crypto_portfolio_volatility)

Crypto Allocation (Asset Class): 0.686451405341143
Equities Weights: [1.]
Crypto Weights: [0.37885162 0.62114838]
Equities Portfolio return 0.21356528492562327
Crypto Portfolio return 0.6281895000832747
Equities Portfolio Volatility 0.27624493303109376
Crypto Portfolio Volatility 0.6047825623453412
